## Install and import the necessary libraries

In [1]:
%%capture

!pip install -r  requirements.txt

In [8]:
import torch
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, GenerationConfig, TextStreamer, pipeline
import pandas as pd
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from typing import List, Tuple
from langchain_core.pydantic_v1 import BaseModel

from langchain.output_parsers import PydanticOutputParser
import os



## Initialize model 

In [3]:
MODEL_NAME = 'gpt-3.5-turbo-instruct'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
model = OpenAI(model_name=MODEL_NAME, openai_api_key="OPENAI_API_KEY", temperature=0)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [7]:

generation_prompt = """
Generate exactly 15 real estate listings as a list of listings from Cox's Bazar neighborhood. All the prices should be in taka, and the location should be in Cox's Bazar, Bangladesh. Here are sample listings enclosed in ticks. Please follow the format and make sure no fields are missing.
Create them in this format:
Neighborhood: Inani Beach, Cox's Bazar
Price (Taka): 
3,500,0
Bedrooms: 3
Bathrooms: 2.5
House Size (sqft): 2,000
Description: Unwind and recharge in this modern 2-bedroom condo, perfectly situated just steps from the pristine shores of Inani Beach. Wake up to the calming sound of waves and enjoy breathtaking ocean views from your private balcony. The light-filled interior features an open floor plan, ideal for entertaining. The sleek kitchen boasts stainless steel appliances and granite countertops, perfect for whipping up gourmet meals. Unwind after a long day on the beach in the spacious living area or take a refreshing dip in the sparkling community pool.
Neighborhood Description: Sea Breeze Lane is a tranquil beachside community offering the perfect blend of relaxation and convenience. Spend your days soaking up the sun on the white sand beaches or explore the vibrant marine life by snorkeling or diving. After a day at the beach, grab a bite at one of the many beachfront cafes or restaurants. Enjoy easy access to local shops, grocery stores, and nightlife options, all within walking distance.
Make sure you generate exactly 15 rows. Ensure the property description aligns with the location and makes it attractive.

Format the with clear titles for each part. Use the same style as the example to list exactly 15 different properties. Make sure you don't generate any less than 15 listings.
"""


/home/student/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [8]:

class RealEstate(BaseModel):
    neighborhood: str
    price: str
    bedrooms: str
    bathrooms: str
    house_size: str
    description: str
    neighborhood_description: str

class ListingCollection(BaseModel):
    listings: List[RealEstate]


parser = PydanticOutputParser(pydantic_object=ListingCollection)

prompt = PromptTemplate(
    template="{instruction}\n{format_instructions}\n",
    input_variables=["instruction"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)



In [9]:
chain = prompt | model | parser
result = chain.invoke({"instruction": generation_prompt})


In [11]:
from fastapi.encoders import jsonable_encoder
import pandas as pd 

listings = jsonable_encoder(result.listings)

df = pd.DataFrame(listings)

df.to_csv("cox_real_estate_listings.csv", index=True)
print("Created cox_real_estate_listings.csv file")

Created cox_real_estate_listings.csv file


In [5]:
df = pd.read_csv('cox_real_estate_listings.csv')
df.head()

,Unnamed: 0,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
0,0,Sea Breeze Lane (near Inani Beach),45000,2,1.5,1000.0,Unwind and recharge in this modern 2-bedroom c...,Sea Breeze Lane offers a tranquil escape just ...
1,1,Himachori Coral Reef (near Himchori Beach),50000,1,1.0,700.0,Embrace a minimalist lifestyle in this chic st...,Himachori Coral Reef offers a unique opportuni...
2,2,Parki Beach Community (near Parki Beach),38000,2,1.0,850.0,Immerse yourself in the vibrant energy of Park...,Parki Beach Community offers a vibrant atmosph...
3,3,Kolatoli Beachfront (near Kolatoli Beach),60000,3,2.0,1500.0,Indulge in luxurious beachfront living in this...,Kolatoli Beachfront offers the epitome of luxu...
4,4,Kongkong Beach Vista (near Kongkong Beach),42000,1,1.0,550.0,Unwind in this cozy studio apartment with a pr...,Kongkong Beach Vista offers a tranquil escape ...


In [6]:
df.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

In [7]:
df.head()

,id,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
0,0,Sea Breeze Lane (near Inani Beach),45000,2,1.5,1000.0,Unwind and recharge in this modern 2-bedroom c...,Sea Breeze Lane offers a tranquil escape just ...
1,1,Himachori Coral Reef (near Himchori Beach),50000,1,1.0,700.0,Embrace a minimalist lifestyle in this chic st...,Himachori Coral Reef offers a unique opportuni...
2,2,Parki Beach Community (near Parki Beach),38000,2,1.0,850.0,Immerse yourself in the vibrant energy of Park...,Parki Beach Community offers a vibrant atmosph...
3,3,Kolatoli Beachfront (near Kolatoli Beach),60000,3,2.0,1500.0,Indulge in luxurious beachfront living in this...,Kolatoli Beachfront offers the epitome of luxu...
4,4,Kongkong Beach Vista (near Kongkong Beach),42000,1,1.0,550.0,Unwind in this cozy studio apartment with a pr...,Kongkong Beach Vista offers a tranquil escape ...


# Prepare our listings for Chromadb
We convert our synthetically generated listings to insert in Chromadb for performing vector search

In [9]:
property_texts = []

property_ids = [{'id': i} for i in range(len(df))]

property_template = """
Description: {}
Neighborhood Description: {}
Price: {}
Bedrooms: {}
Bathrooms: {}
House Size (sqft): {}
"""

for index, row in df.iterrows():
    property_texts.append(property_template.format(
        row['Description'],
        row['Neighborhood Description'],
        row['Price'],
        row['Bedrooms'],
        row['Bathrooms'],
        row['House Size']
    ))


In [14]:
property_texts

["\nDescription: Unwind and recharge in this modern 2-bedroom condo, a stone's throw from the pristine shores of Inani Beach. Listen to the calming sound of waves from your private balcony and enjoy breathtaking ocean views. The light-filled interior features an open floor plan, perfect for entertaining. Prepare gourmet meals in the sleek kitchen with stainless steel appliances and granite countertops. Dive into the sparkling community pool after a long day at the beach. Embrace the relaxed beach lifestyle in Sea Breeze Lane.\nNeighborhood Description: Sea Breeze Lane offers a tranquil escape just steps away from the vibrant Inani Beach. Explore the diverse marine life by snorkeling or diving, or simply soak up the sun on the white sand beaches. Enjoy a variety of restaurants and cafes within walking distance, or head to the local shops for your essentials. This community is perfect for young professionals seeking a peaceful haven close to the beach.\nPrice: 45000\nBedrooms: 2\nBathroo

## Create a collection

In [10]:

db = Chroma(
    collection_name="cox_listings", embedding_function=OpenCLIPEmbeddings()
)


open_clip_pytorch_model.bin:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

# Insert the listings

In [11]:
db.add_texts(texts=property_texts, metadatas = property_ids)

['65ec7236-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec72f4-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec7326-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec734e-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec736c-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec738a-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec73b2-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec73d0-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec73ee-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec740c-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec742a-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec7448-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec7470-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec7498-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec74b6-24f9-11ef-a6d8-16ffd57d1cf7',
 '65ec74d4-24f9-11ef-a6d8-16ffd57d1cf7']

# Perform semantic search and personalize the responses

In [12]:
def get_matched_listings(user_preferences: List[str], top_k: int = 5) -> List[int]:

    combined_preferences = '\n'.join(user_preferences)
    results = db.similarity_search(combined_preferences, k=top_k * 2)

    return [result.metadata['id'] for result in results if result.metadata['id'] not in results[:top_k]]


## Add a new prompt template for custom listing generation
We will use in-context learning to customize the listings based on user preferences. To do so, we will create a new prompt template
that will take an `instruction` and `buyer preferences` as context. Then will use the `buyer preferences` to retrieve relevant 
listings and augment the `instruction`, `preference` and relevant `listings` to query an llms to create a new personalized listing. 

## Create instruction

In [13]:
instructions = 'Create a short description tailored to each listing, capturing the kind of what the buyer desires and emphasizing those key features to attract their interest.'

## Create prompt template with validation

In [17]:
class ListingSummary(BaseModel):
    listing_id: int
    summary_text: str

class SummaryCollection(BaseModel):
    listing_summaries: List[ListingSummary]


parser = PydanticOutputParser(pydantic_object=SummaryCollection)
prompt_template = PromptTemplate(
    template="{instruction}\nBuyer Preferences:\n{buyer_preferences}\nListings:\n{listings}\n\n{format_instructions}",
    input_variables=["instruction", "buyer_preferences", "listings"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)


## Create function to retrieve relvant listings and augment them with preferences and instruction to get customized listings

In [18]:
cached_summaries = {}

def generate_customized_summaries(buyer_preferences: List[str], top_k: int = 5) -> List[Tuple[int, str]]:

    top_listings = get_matched_listings(buyer_preferences, top_k)

    listings = [('ID:' + str(listing_id), property_texts[listing_id]) for listing_id in top_listings]

    query = prompt_template.format(
        instruction=instructions,
        buyer_preferences='\n'.join(buyer_preferences),
        listings='\n'.join([''.join(listing) for listing in listings])
    )

    response = []
    for summary in parser.parse(model(query)).listing_summaries:
        response.append((summary.listing_id, summary.summary_text))
        cached_summaries[summary.listing_id] = summary.summary_text

    return response


## Test out our personalized listings generator

In [19]:
questions = ["What type of neighborhood are you aiming for?",
             "Which house size best suits your needs?",
             "How many bed do you want?",
             "How many bath do you want?",
             "What is the price range you want?"]

preferences = [
        "I'm targeting peaceful communities near popular beaches in Cox's Bazar, offering a relaxed and scenic atmosphere."
        "The available house sizes should range from 500 to 1500 square feet, a medium sized place is best for me.",
        "I am looking for ideally a three bedroom space",
        "The house needs to have two bathrooms, one for personal and one for guests",
        "My budget is around 30000 thousand for rent, so rent should be within 20000 to 30000 "
]


In [20]:
result = generate_customized_summaries(preferences)
print(result)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[(8, 'Embrace the panoramic views in this modern studio apartment located in the Dune View Apartments near Chanderia Beach. The light-filled space features floor-to-ceiling windows showcasing breathtaking vistas of the rolling sand dunes and the sparkling ocean beyond. The well-equipped kitchenette allows you to prepare quick meals, while the spacious balcony provides the perfect spot for enjoying the sunset. Take a short walk down to Chanderia Beach, known for its pristine sands and calm waters. Explore the nearby shops and restaurants or simply relax and soak up the sun. This affordable community offers a perfect retreat for young professionals seeking a convenient beach lifestyle with stunning views.'), (15, "Immerse yourself in the local culture in this cozy studio apartment located in the Fisherman's Wharf Apartments near Chanderia Beach. This light-filled space features a balcony overlooking the bustling harbor, offering a glimpse into the daily lives of local fishermen. The well

In [21]:
preferences2 = [
  "I'm seeking an eco-friendly community close to the beach, with access to sustainability initiatives and a focus on responsible living.",
  "A studio or one-bedroom apartment (around 600-800 square feet) would be ideal for my minimalist lifestyle.",
  "While I can manage with one bathroom, having 1.5 bathrooms would provide extra convenience, especially for guests.",
  "Budget is a key factor, with my ideal range being between 35,000 taka and 45,000 taka per month.",
  "Proximity to shops, restaurants, and cafes is important for me, as I enjoy a vibrant and convenient beach lifestyle."
]
result = generate_customized_summaries(preferences2)
print(result)

[(12, 'Find your peace in this cozy studio apartment nestled in the Hidden Cove Retreat near Inani Beach. This cleverly designed space maximizes comfort, featuring a well-equipped kitchenette, a comfortable sleeping area with ample natural light, and a modern bathroom. Step out onto your private balcony and enjoy the tranquility of a hidden cove accessible only by a short walk through lush greenery. Take a refreshing dip in the crystal-clear waters of a secluded beach and explore the diverse marine life with minimal crowds. This budget-friendly community offers a perfect escape for young professionals seeking a peaceful and secluded haven near the beach.'), (1, 'Embrace a minimalist lifestyle in this chic studio apartment, boasting stunning views of the coral reefs just off Himchori Beach. This cleverly designed space maximizes functionality, featuring a well-equipped kitchen, a comfortable sleeping area with ample natural light, and a stylish bathroom. Step out onto your private balco